<a href="https://colab.research.google.com/github/ashwin-yedte/visual-intelligence-travel-finance/blob/main/backend_api_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VLM INTELLIGENCE LAYER


UNIFIED BACKEND API


Complete pipeline: Steps 1-2-3-4 in single API call

Run this AFTER loading all Step 1-4 cells

# =================================================================
PREREQUISITES
# =================================================================

BEFORE RUNNING THIS:
1. Run ALL cells from Step1_Production_Clean.py
2. Run ALL cells from Step2_Destination_Matching.py
3. Run ALL cells from Step3_Theme_Aggregation.py
4. Run ALL cells from Step4_Recommended_Destinations.py
5. Set ngrok auth token
6. Start server with start_complete_server()


# =================================================================
Step 1: IMPORTS
# =================================================================


In [ ]:
print("="*80)
print("UNIFIED BACKEND API - VLM INTELLIGENCE LAYER")
print("="*80)

# Install pyngrok if not already installed
!pip install pyngrok

from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from typing import List
import uvicorn
from pyngrok import ngrok
from threading import Thread

print("Imports complete")
print("="*80)

UNIFIED BACKEND API - VLM INTELLIGENCE LAYER
Imports complete


# =================================================================
Step 2: CREATE FASTAPI APP
# =================================================================


In [ ]:
app = FastAPI(
    title="VLM Intelligence API ",
    description="Visual travel destination recommendation system",
    version="1.0"
)

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

print("="*80)
print("FastAPI app created")
print("="*80)


FastAPI app created


# =================================================================
Step 3: COMPLETE PIPELINE ENDPOINT
# =================================================================


In [ ]:
@app.get("/")
async def root():
    """Health check endpoint"""
    return {
        "status": "online",
        "service": "VLM Intelligence API - Complete Pipeline",
        "version": "1.0",
        "pipeline": "Steps 1-2-3-4",
        "endpoints": {
            "health": "GET /",
            "complete_pipeline": "POST /api/recommend-destinations"
        }
    }

@app.post("/api/recommend-destinations")
async def recommend_destinations_complete(files: List[UploadFile] = File(...)):
    """
    COMPLETE PIPELINE: Steps 1-2-3-4

    User uploads images -> Returns recommended destinations

    Pipeline:
    1. Image Analysis (validation, CLIP embeddings)
    2. Destination Matching (per-image top-10)
    3. Theme Aggregation (majority vote, re-ranking)
    4. Recommendations (enrich with metadata)

    Returns:
        JSON with user profile and top 10 recommended destinations
    """

    try:
        print("\n" + "="*80)
        print("COMPLETE PIPELINE REQUEST")
        print("="*80)
        print("Files received: " + str(len(files)))

        # ==================================================================
        # STEP 1: IMAGE ANALYSIS
        # ==================================================================

        print("\n" + "-"*80)
        print("STEP 1: IMAGE ANALYSIS")
        print("-"*80)

        # Convert files to dict
        files_dict = {}
        for file in files:
            image_bytes = await file.read()
            files_dict[file.filename] = image_bytes

        # Analyze images
        step1_result = analyze_user_images(files_dict)

        if step1_result['status'] == 'error':
            return JSONResponse(
                status_code=400,
                content={
                    "status": "error",
                    "step": "step1_image_analysis",
                    "error": step1_result.get('error'),
                    "error_code": step1_result.get('error_code'),
                    "validation_errors": step1_result.get('validation_errors', [])
                }
            )

        save_step1_outputs(step1_result)
        print("Step 1 complete: " + str(step1_result['num_processed']) + " images processed")

        # ==================================================================
        # STEP 2: DESTINATION MATCHING
        # ==================================================================

        print("\n" + "-"*80)
        print("STEP 2: DESTINATION MATCHING")
        print("-"*80)

        import numpy as np
        user_embeddings = np.array(step1_result['embeddings'])

        step2_result = match_destinations_per_image(user_embeddings)
        save_step2_outputs(step2_result)
        print("Step 2 complete: " + str(step2_result['total_unique_destinations']) + " destinations matched")

        # ==================================================================
        # STEP 3: THEME AGGREGATION
        # ==================================================================

        print("\n" + "-"*80)
        print("STEP 3: THEME AGGREGATION")
        print("-"*80)

        per_image_matches = step2_result['per_image_matches']

        theme_analysis = analyze_theme_distribution(per_image_matches)
        destination_data = aggregate_destination_data(per_image_matches)
        ranked_destinations = rerank_destinations(destination_data, theme_analysis)

        save_step3_outputs(theme_analysis, ranked_destinations)
        print("Step 3 complete: Dominant theme is " + theme_analysis['dominant_theme'])

        # ==================================================================
        # STEP 4: RECOMMENDED DESTINATIONS
        # ==================================================================

        print("\n" + "-"*80)
        print("STEP 4: RECOMMENDED DESTINATIONS")
        print("-"*80)

        recommendations_data = build_recommendations(ranked_destinations, theme_analysis)
        save_step4_outputs(recommendations_data)
        print("Step 4 complete: " + str(recommendations_data['total_recommendations']) + " recommendations ready")

        # ==================================================================
        # PREPARE RESPONSE
        # ==================================================================

        print("\n" + "="*80)
        print("PIPELINE COMPLETE")
        print("="*80)

        response = {
            "status": "success",
            "pipeline_summary": {
                "images_uploaded": step1_result['num_uploaded'],
                "images_processed": step1_result['num_processed'],
                "destinations_matched": step2_result['total_unique_destinations'],
                "dominant_theme": theme_analysis['dominant_theme'],
                "theme_confidence": round(theme_analysis['theme_confidence'] * 100, 2)
            },
            "user_profile": recommendations_data['user_profile'],
            "recommendations": recommendations_data['recommendations']
        }

        return JSONResponse(content=response)

    except NameError as e:
        return JSONResponse(
            status_code=500,
            content={
                "status": "error",
                "error": "Pipeline functions not loaded. Did you run all Step 1-4 cells?",
                "error_code": "PIPELINE_NOT_LOADED",
                "details": str(e)
            }
        )

    except Exception as e:
        import traceback
        return JSONResponse(
            status_code=500,
            content={
                "status": "error",
                "error": "Server error: " + str(e),
                "error_code": "SERVER_ERROR",
                "traceback": traceback.format_exc()
            }
        )


print("Complete pipeline endpoint defined")
print("="*80)



Complete pipeline endpoint defined


# =================================================================
Step 4: VERIFY ALL STEPS LOADED
# =================================================================


In [ ]:
def verify_pipeline_loaded():
    """Check if all Steps 1-4 are loaded and ready."""

    print("\n" + "="*80)
    print("VERIFYING COMPLETE PIPELINE")
    print("="*80)

    required_functions = [
        ('analyze_user_images', 'Step 1'),
        ('save_step1_outputs', 'Step 1'),
        ('match_destinations_per_image', 'Step 2'),
        ('save_step2_outputs', 'Step 2'),
        ('analyze_theme_distribution', 'Step 3'),
        ('aggregate_destination_data', 'Step 3'),
        ('rerank_destinations', 'Step 3'),
        ('save_step3_outputs', 'Step 3'),
        ('build_recommendations', 'Step 4'),
        ('save_step4_outputs', 'Step 4')
    ]

    all_ok = True

    for func_name, step in required_functions:
        try:
            func = globals()[func_name]
            print("  Found " + func_name + " (" + step + ")")
        except KeyError:
            print("  MISSING: " + func_name + " (" + step + ")")
            all_ok = False

    print("="*80)

    if all_ok:
        print("SUCCESS: Complete pipeline loaded and ready")
    else:
        print("ERROR: Some pipeline functions missing")
        print("Solution: Run all cells from Steps 1-4 first")

    print("="*80)
    return all_ok



# ==================================================================
Step 5: START SERVER
# ==================================================================


In [ ]:
def start_complete_server():
    """Start complete pipeline server with ngrok."""

    if not verify_pipeline_loaded():
        print("\nCANNOT START SERVER")
        print("Please run all Step 1-4 cells first")
        return

    print("\n" + "="*80)
    print("STARTING COMPLETE PIPELINE SERVER")
    print("="*80)

    # Start ngrok
    print("Creating ngrok tunnel...")
    ngrok_tunnel = ngrok.connect(8000)
    public_url = ngrok_tunnel.public_url

    print("\nSERVER IS RUNNING")
    print("="*80)
    print("Public URL: " + public_url)
    print("\nFrontend should use:")
    print("  " + public_url + "/api/recommend-destinations")
    print("\nEndpoints:")
    print("  GET  " + public_url + "/")
    print("  POST " + public_url + "/api/recommend-destinations")
    print("\nPipeline: Steps 1-2-3-4 (complete)")
    print("\nTo stop: Runtime -> Interrupt execution")
    print("="*80)

    # Run uvicorn
    def run_uvicorn():
        uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

    thread = Thread(target=run_uvicorn, daemon=True)
    thread.start()

    try:
        thread.join()
    except KeyboardInterrupt:
        print("\nServer stopped")


print("\nComplete backend API ready")
print("="*80)
print("\nTO START SERVER:")
print("1. Verify pipeline: verify_pipeline_loaded()")
print("2. Set ngrok auth: ngrok.set_auth_token('YOUR_TOKEN')")
print("3. Start server: start_complete_server()")
print("="*80)


Complete backend API ready

TO START SERVER:
1. Verify pipeline: verify_pipeline_loaded()
2. Set ngrok auth: ngrok.set_auth_token('YOUR_TOKEN')
3. Start server: start_complete_server()


In [ ]:
verify_pipeline_loaded()


VERIFYING COMPLETE PIPELINE
  MISSING: analyze_user_images (Step 1)
  MISSING: save_step1_outputs (Step 1)
  MISSING: match_destinations_per_image (Step 2)
  MISSING: save_step2_outputs (Step 2)
  MISSING: analyze_theme_distribution (Step 3)
  MISSING: aggregate_destination_data (Step 3)
  MISSING: rerank_destinations (Step 3)
  MISSING: save_step3_outputs (Step 3)
  MISSING: build_recommendations (Step 4)
  MISSING: save_step4_outputs (Step 4)
ERROR: Some pipeline functions missing
Solution: Run all cells from Steps 1-4 first


False